In [ ]:
# Veja mais informações no post do Medium

# https://medium.com/@conversandoemcodigos/resolvendo-o-letreco-de-maneira-aut%C3%B4noma-76373fcfbd36

In [ ]:
#Importando as bibliotecas. 
# Para usar o Selenium, é necessário baixar também o drive compatível

import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pyautogui
from time import sleep
from random import randint

In [ ]:
# Essa define a cor da letra.
def cor_letra(x, letra):
    if len(verde[x]) == 0:
        if letra == 'letter-wrapper rounded d-flex justify-content-center align-items-center right me-2' or letra == 'letter-wrapper rounded d-flex justify-content-center align-items-center right':
            verde[x].insert(x, palavra_digitada[x])
        elif letra == "letter-wrapper rounded d-flex justify-content-center align-items-center displaced" or letra== 'letter-wrapper rounded d-flex justify-content-center align-items-center displaced me-2':
            amarelo[x].insert(x, palavra_digitada[x])
        elif letra == "letter-wrapper rounded d-flex justify-content-center align-items-center wrong" or letra== "letter-wrapper rounded d-flex justify-content-center align-items-center wrong me-2":
            vermelho[x].insert(x, palavra_digitada[x])

        
# Essa função define em qual linha do site vai ser trabalhado e depois usa função anterior para definir a cor da letra      
def fila(linha):
    letra0 = navegador.find_element_by_xpath(f'//*[@id="root"]/div/div[2]/div[1]/div/div/div[{linha}]/div[1]').get_attribute('class')
    letra1 = navegador.find_element_by_xpath(f'//*[@id="root"]/div/div[2]/div[1]/div/div/div[{linha}]/div[2]').get_attribute('class')
    letra2 = navegador.find_element_by_xpath(f'//*[@id="root"]/div/div[2]/div[1]/div/div/div[{linha}]/div[3]').get_attribute('class')
    letra3 = navegador.find_element_by_xpath(f'//*[@id="root"]/div/div[2]/div[1]/div/div/div[{linha}]/div[4]').get_attribute('class')
    letra4 = navegador.find_element_by_xpath(f'//*[@id="root"]/div/div[2]/div[1]/div/div/div[{linha}]/div[5]').get_attribute('class')
    cor_letra(0, letra0)
    cor_letra(1, letra1)
    cor_letra(2, letra2)
    cor_letra(3, letra3)
    cor_letra(4, letra4)
    

In [ ]:
# Estabelecendo as listas
verde = [[],[],[],[],[]]
vermelho = [[],[],[],[],[]]
amarelo = [[],[],[],[],[]]

# subindo o banco de palavras
palavras = pd.read_csv('termo.csv')

# Solicitando a palavra
palavra_digitada = input('Digite uma palavra de cinco letras: ')

# Entrando no site e digitando a palavra escolhida
navegador = webdriver.Chrome()
navegador.get('https://www.gabtoschi.com/letreco/')
sleep(1)
pyautogui.write(palavra_digitada)
navegador.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[2]/div[1]/button[2]').click()

# Mudando de fila automaticamente
for num in range(1, 6):
    fila(num)
    for c in range(0, 5):
        
        # Verde: elimina primeiro todas as palavras que não tem a letra verde
        # e depois elimina as que não tem a letra verde naquela posição 

        if len(verde[c]) == 1:
            palavras = palavras[palavras['palavras'].str.contains(verde[c][0])]
            for i in palavras.index:
                if palavras['palavras'][i][c] != verde[c][0]:
                    palavras = palavras.drop(i)
        
        # Amarelo: primeiro elimina quem não tem a letra amarela e depois apaga
        # quem tem a letra naquela posição

        elif len(amarelo[c]) == 1:
            palavras = palavras[palavras['palavras'].str.contains(amarelo[c][0])]
            for i in palavras.index:
                if palavras['palavras'][i][c] == amarelo[c][0]:
                    palavras = palavras.drop(i)
                    
        # Vermelho: elimina as palavras que tem a letra naquela posição  
        
        elif len(vermelho[c]) == 1:
            for i in palavras.index:
                if palavras['palavras'][i][c] == vermelho[c][0]:
                    palavras = palavras.drop(i)
                    
    # Limpeza: 
    vermelho = [[],[],[],[],[]]
    amarelo = [[],[],[],[],[]]
                    
    # Resetando o índice e gerando uma nova palavra para ser digitada 
    palavras.reset_index(drop=True, inplace=True)
    sorteio = randint(0, len(palavras) - 1)
    palavra_digitada = palavras['palavras'][sorteio]       
    sleep(1)
    pyautogui.write(palavra_digitada)
    navegador.find_element_by_xpath('//*[@id="root"]/div/div[2]/div[2]/div[1]/button[2]').click()

In [ ]:
# Para encontrar qual o indíce da palavra a se excluir

palavras = pd.read_csv('termo.csv')
palavras.query('palavras == "_____"')

In [ ]:
# Exclui a palavra e exporta um novo dicionário

palavras = palavras.drop()
palavras.to_csv('termo.csv', index=False)